<a href="https://colab.research.google.com/github/Shubbair/GPT4-Tokenizer/blob/main/GPT4Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use *Shakespear Poem*

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-11 13:41:56--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-03-11 13:41:56 (22.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.3 MB/s eta 0:00:00


## Import Libraries

In [51]:
import tiktoken
import unicodedata
import regex as re
from tqdm import tqdm

#### set GPT4 patten to apply regex on it

In [5]:
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

<h4>BPE (Byte Pair Encoding)</h4>

In [52]:
# helper function used in get_gpt4_merges() to reconstruct the merges
def bpe(mergeable_ranks : dict[bytes, int], token : list, max_rank : int) -> list:
    parts = [bytes([b]) for b in token]
    while True:
        min_idx = None
        min_rank = None
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair[0] + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank
        if min_rank is None or (max_rank is not None and min_rank >= max_rank):
            break
        assert min_idx is not None
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2:]
    return parts

# get the merges from the gpt4
def recover_merges(mergeable_ranks : dict[bytes, int]) -> list:
    # the `merges` are already the byte sequences in their merged state.
    # so we have to recover the original pairings. We can do this by doing
    # a small BPE training run on all the tokens, in their order.
    merges = {}
    for token, rank in mergeable_ranks.items():
        if len(token) == 1:
            continue # skip raw bytes
        pair = tuple(bpe(mergeable_ranks, token, max_rank=rank))
        assert len(pair) == 2
        # recover the integer ranks of the pair
        ix0 = mergeable_ranks[pair[0]]
        ix1 = mergeable_ranks[pair[1]]
        merges[(ix0, ix1)] = rank

    return merges

# return counts of pairs
def get_stats(ids : list[int]) -> list:
    counts = {}
    for pair in zip(ids, ids[1:]): # take two pairs of
        counts[pair] = counts.get(pair, 0) + 1
    return counts

# merge pair in ids with new idx
def merge(ids : list[int], pair, idx) -> list:
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

def replace_control_characters(s: str) -> str:
    # replace control chars
    # Control characters -> http://www.unicode.org/reports/tr44/#GC_Values_Table
    chars = []
    for ch in s:
        if unicodedata.category(ch)[0] != "C":
            chars.append(ch) # this character is ok
        else:
            chars.append(f"\\u{ord(ch):04x}") # escape
    return "".join(chars)


def render_token(t: bytes) -> str:
    # pretty print a token, escaping control characters
    s = t.decode('utf-8', errors='replace')
    s = replace_control_characters(s)
    return s

In [53]:
class GPT4Tokenizer:
    def __init__(self):
        self.pattern = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
        enc = tiktoken.get_encoding("cl100k_base")
        self.mergeable_ranks = enc._mergeable_ranks

        print('Number of tokens:',len(self.mergeable_ranks))

        self.merges = recover_merges(self.mergeable_ranks)

        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        self.vocab = vocab

        # shuffle values
        self.byte_shuffle = {i: self.mergeable_ranks[bytes([i])] for i in range(256)}
        # return index , value
        self.inverse_byte_shuffle = {v: k for k, v in self.byte_shuffle.items()}

    def encode_chunk(self,text_bytes : list[bytes],merges : dict[tuple,int]) -> list[int]:
        # before we start processing bytes, we have to permute them
        text_bytes = bytes(self.byte_shuffle[b] for b in text_bytes)
        ids = list(text_bytes)
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = merges[pair]
            ids = merge(ids, pair, idx)
        return ids

    def encode(self,text: str) -> list[int]:
        text_chunks = re.findall(self.pattern, text)
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8")
            chunk_ids = self.encode_chunk(chunk_bytes,self.merges)
            ids.extend(chunk_ids)
        return ids

    def decode(self,ids:int)-> str:
      # we have to un-permute the bytes before we decode
      text_bytes = b"".join(self.vocab[idx] for idx in ids)
      text_bytes = bytes(self.inverse_byte_shuffle[b] for b in text_bytes)
      text = text_bytes.decode("utf-8", errors="replace")
      return text

    def bpe_encode(self,mergeable_ranks: dict[bytes, int], input: bytes) -> list[int]:
        parts = [bytes([b]) for b in input]
        while True:
            # Iterate over all pairs and find the pair we want to merge the most
            min_idx = None
            min_rank = None
            for i, pair in enumerate(zip(parts[:-1], parts[1:])):
                rank = mergeable_ranks.get(pair[0] + pair[1])
                if rank is not None and (min_rank is None or rank < min_rank):
                    min_idx = i
                    min_rank = rank

            # If there were no pairs we could merge, we're done!
            if min_rank is None:
                break
            assert min_idx is not None

            # Otherwise, merge that pair and leave the rest unchanged. Then repeat.
            parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2 :]
        return parts

    def visualise_tokens(self,text: str) -> None:
      # tokenize the plain text
      pat = re.compile(gpt4.pattern)
      words = pat.findall(txt)
      tokens = []
      for word in words:
          # Turn each word into tokens, using the byte pair encoding algorithm
          word_bytes = word.encode("utf-8")
          word_tokens = self.bpe_encode(self.mergeable_ranks, word_bytes)
          tokens.extend(word_tokens)

      background = [f"\u001b[48;5;{i}m" for i in [167, 179, 185, 77, 80, 68, 134]]
      # If token boundaries do not occur at unicode character boundaries, it's unclear how best to
      # visualise the token. Here, we'll just use the unicode replacement character to represent some
      # fraction of a character.
      unicode_token_values = [x.decode("utf-8", errors="replace") for x in tokens]
      print(unicode_token_values)
      running_length = 0
      last_color = None
      for token in unicode_token_values:
          color = background[running_length % len(background)]
          if color == last_color:
              color = background[(running_length + 1) % len(background)]
              assert color != last_color
          last_color = color
          running_length += len(token)
          print(color + token, end="")
      print("\u001b[0m")

    def save_vocab(self, vocab_file : str) -> None:
      # build vocab being mindful of the byte shuffle
      vocab = {idx: bytes([self.inverse_byte_shuffle[idx]]) for idx in range(256)}
      for (p0, p1), idx in self.merges.items():
          vocab[idx] = vocab[p0] + vocab[p1]
      # now merge the shuffled bytes and write to file
      inverted_merges = {idx: pair for pair, idx in self.merges.items()}
      with open(vocab_file, "w", encoding="utf-8") as f:
          for idx, token in vocab.items():
              s = render_token(token)
              if idx in inverted_merges:
                  idx0, idx1 = inverted_merges[idx]
                  s0 = render_token(vocab[idx0])
                  s1 = render_token(vocab[idx1])
                  f.write(f"[{s0}][{s1}] -> [{s}] {idx}\n")
              else:
                  f.write(f"[{s}] {idx}\n")

In [55]:
gpt4 = GPT4Tokenizer()

Number of tokens: 100256


In [56]:
txt = 'iveodeost class not'

In [57]:
gpt4.visualise_tokens(txt)

['ive', 'ode', 'ost', ' class', ' not']
iveodeost class not


In [58]:
gpt4.save_vocab('gpt4.vocab')

#### test on karpathy test file : https://github.com/karpathy/minbpe/blob/master/tests/test_tokenizer.py

In [31]:
enc = tiktoken.get_encoding("cl100k_base")
tiktoken_ids = enc.encode(text)

In [32]:
gpt4_tokenizer_ids = gpt4.encode(text)

In [36]:
'Same' if gpt4_tokenizer_ids == tiktoken_ids else 'Not Same'

'Same'

# The Issue with GPT4 the tokenizer due to the process of merging tokens there are ambiguous tokens like :
Token: ' attRot' </br>                    
Token: '�' </br>  
Token: 'EStreamFrame' </br>   
Token: ' SolidGoldMagikarp' </br>  
